In [4]:
# 기본 세팅

from urllib.request import Request, urlopen
from urllib.parse import quote, urlencode, quote_plus, unquote
import json
from bs4 import BeautifulSoup as bs


import copy
import os
import datetime as dt
import time as t
import numpy as np
import pandas as pd
import random as rd

#from tqdm.notebook import tqdm as tq



# 날짜 지정

In [267]:
'''1. 목표 날짜 지정  (검색할 날짜)'''
# 검색 시작 날짜    
first_date = 20200201
current_date = (int(str(first_date)[:4]) , int(str(first_date)[4:6]), int(str(first_date)[6:]))

# 검색 종료 날짜
end_date = 20200205


# 크롤링

In [271]:
########################################
##### 1. url 크롤링                #####
########################################




##############
data_set = {}          # 데이터가 담길 dictionary
############## 




# first_date 에서 날짜 넘겨받는 부분  (시작점)
current_date = (int(str(first_date)[:4]) , int(str(first_date)[4:6]), int(str(first_date)[6:]))
current_date_WHILE = int(dt.datetime(current_date[0], current_date[1], current_date[2]).strftime('%Y%m%d'))




'''1. 날짜 하루씩 추가하면서 크롤링 진행'''
# '날짜 지정'에서 설정한 검색종료 날짜까지 작업 반복
while current_date_WHILE <=end_date :
    print(current_date_WHILE)
    



    '''
    2. current_date 를 기반으로 날짜 변수를 datetime 형태로 저장  -
    ->    이후의 날짜 계산을 용이하게 하기 위함
    '''
    s_date = dt.datetime(current_date[0], current_date[1], current_date[2]) 
    e_date = s_date + dt.timedelta(days=1)   


    '''
    3. datetime 날짜의 type 조작   -->    검색창에 입력할 수 있는 형태로 변환
    '''

    # 1.  20200201  형태로 저장
    str_s_date1 = s_date.strftime('%Y%m%d')
    str_e_date1 = e_date.strftime('%Y%m%d')

    # 2,  2020.02.01 형태로 저장
    str_s_date2 = s_date.strftime('%Y.%m.%d')
    str_e_date2 = e_date.strftime('%Y.%m.%d')


    '''
    4. url 및 검색 parameter 지정
    '''

    # base url
    base_url = 'https://search.naver.com/search.naver'

    # parameters
    query_params = '?' + urlencode(
        {
            quote_plus('where') : 'news',
            quote_plus('query') : "+코로나 창업 | 스타트업",
            quote_plus('sm') : 'tab_opt',
            quote_plus('') : '',
            quote_plus('sort')  : '2',
            quote_plus('photo') : '0',
            quote_plus('field') : '0',
            quote_plus('pd') : '3',
            quote_plus('ds') : str_s_date2,
            quote_plus('de') : str_e_date2,
            quote_plus('docid') : '',  
            quote_plus('related') : '0',
            quote_plus('mynews') : '1',       # 언론사 선택시 1 / default = 0 
            quote_plus('office_type') :  '1',  # default = 0 / 일간지 = 1 
            quote_plus('office_section_code') : '1',  # 언론사 선택하면 1, default = 0
            
            ########################
            ### 신문사 코드 입력 ###
            ########################
            
            quote_plus('news_office_checked') : '1023',  #중앙일보 1023,  만일 없으면 공백으로 비워두어야 함
        }
    )




    # 첫번째 반복시행에서 article index는 1 고정
    first_article_index = 1




    ###############################  특정 날짜에서의 검색창 크롤링 ###########################
    #iteration_idx = 0
    while True:    

        
        try:
            #debug code
            #iteration_idx +=1
            #print('while 작업 횟수 : ', iteration_idx)

            '''
            5. 뉴스 검색창 접속
            '''
            # 기타 parameter
            start = '&start=' + str(first_article_index)  # 페이지 첫번째 기사의 index, 
                                    # 처음은 1로 시작하나 페이지 넘길 때마다 10씩 증가시켜야 함
                                    # 즉, 날짜별 iteration 코드에 < start += 10 >  를 넣어주어야 함
            query_others = '&nso=so%3Ar%2Cp%3Afrom' + str_s_date1 + 'to' + str_e_date1 + '&is_sug_officeid=0' + start

            # url
            url = base_url + query_params + query_others
            # 서버에 접속. 너무 급한 속도의 접속은 IP ban을 유발할 수 있음
            req = Request(url, headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'}
            )
            page = urlopen(req)
            
        except:
            t.sleep(20)



            '''
            5. 뉴스 검색창 접속
            '''
            # 기타 parameter
            start = '&start=' + str(first_article_index)  # 페이지 첫번째 기사의 index, 
                                    # 처음은 1로 시작하나 페이지 넘길 때마다 10씩 증가시켜야 함
                                    # 즉, 날짜별 iteration 코드에 < start += 10 >  를 넣어주어야 함
            query_others = '&nso=so%3Ar%2Cp%3Afrom' + str_s_date1 + 'to' + str_e_date1 + '&is_sug_officeid=0' + start

            # url
            url = base_url + query_params + query_others
            # 서버에 접속. 너무 급한 속도의 접속은 IP ban을 유발할 수 있음
            req = Request(url, headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'}
            )
            page = urlopen(req)

            
            
        t.sleep(0.3)     # IP ban 방지용
        

        
        
        '''
        6. page parsing
        '''
        
        # bs parsing
        page = bs(page, 'html.parser')

        # IS_EMPTY 는 검색결과가 있다면 [] 를 반환함. 
        # 만일 IS_EMPTY 가 []가 아니라면 검색결과가 없는 것이므로 break 해야 함.
        IS_EMPTY = page.find_all('div', 'api_noresult_wrap')
        if IS_EMPTY != []:
            break 

        # 필요 데이터 parsing (url, 날짜)
        url_resultset = page.find_all('div', 'news_info')   #  --> url
        news_date_resultset = page.find_all('div', 'info_group')  # --> 날짜,  둘다 parsing 후의 index는 같음



        '''
        7. 지면뉴스 오류 handling
        '''
        for idx in range(len(url_resultset)):

            # url
            target_url = url_resultset[idx].a['data-url']

            # 날짜
            if news_date_resultset[idx].i:
                IS_PAPERNEWS = True
            else:
                IS_PAPERNEWS = False

            if IS_PAPERNEWS ==False:
                target_date = news_date_resultset[idx].find('span', 'info').text

            else:
                try:
                    target_date = news_date_resultset[idx].find_all('span', 'info')[1].text
                except:
                    target_date = news_date_resultset[idx].find_all('span', 'info')[0].text


            # 데이터 저장, key = url  / value = date
            data_set[target_url] = target_date




        # 다음 검색 페이지로 넘어가기 위해 기사 index 조정
        first_article_index += 10




    # 반복문 break에 사용할 current_date 설정
    current_date = s_date
    current_date += dt.timedelta(days = 1)  # 하루 더하기
    current_date = current_date.strftime('%Y%m%d')  # 더한 날짜정보를 str으로 바꾸기

    # str으로 변환된 날짜정보를 연,월,일로 분리
    year = current_date[:4]
    month = current_date[4:6]
    day = current_date[6:]

    # 다음 iteration에 넘어갈 current_date 정보
    current_date = (int(year), int(month), int(day))


    # 날짜 while문 종료에 관한 정보
    current_date_WHILE = int(dt.datetime(current_date[0], current_date[1], current_date[2]).strftime('%Y%m%d'))

    
    
    
    
    
    
'''
데이터 내보내기 (url)
'''     
        
table = pd.DataFrame(data_set.keys(), data_set.values())
table = table.drop_duplicates()
table.to_csv('중앙일보 url주소.csv', encoding = 'cp949')




########################################
##### 2. 본문 크롤링               #####
########################################



# url 가져오기 (csv파일)

urls = list(pd.read_csv('중앙일보 url주소.csv', encoding = 'cp949')['0'])




# 데이터가 들어갈 공간
data_set = {}


'''
data columns :
    date   / title  / body   / company  / url
    (날짜) / (제목) / (본문) / (언론사) / (url)

'''


#url 접속 후 크롤링


for idx in range(len(urls)):
    
    # 차단 막기 위해 잠시 휴식
    #t.sleep(0.3)
    
    # 초기화
    body = ''
    title = ''
    day = ''
    company = ''

    
    # 중앙일보
    if 'chosun.com' in urls[idx]:
        
        
        # '여성조선'은 구현하지 않음
        if 'woman.chosun.com' in urls[idx]:
            continue
        # 'IT 조선'은 구현하지 않음
        if 'it.chosun.com' in urls[idx]:
            continue
        # 'tv조선'은 구현하지 않음
        if 'tvchosun.com' in urls[idx]:
            continue
        
        
        
        ### 접속 ###
        try: 
            req = Request(urls[idx], headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'}
            )
            t.sleep(0.3)
            page = urlopen(req)
            page = bs(page, 'html.parser')

        except:
            # RemoteDiscunnected exception handling
            t.sleep(20)
            req = Request(urls[idx], headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'}
            )
            t.sleep(0.3)
            page = urlopen(req)
            page = bs(page, 'html.parser')
            
            
            
        ### parsing ###
        resultset = page.find_all()[0].decode_contents().split('_id')
        temp = []
        for line in resultset:
            if ('content' in line) and ('"type":"text"' in line) :
                # 기사 head에 해당하는 부분이므로 제거
                if 'strong' in line:
                    continue
                # 본문 부분만 긁어서 한 문장으로 합치기
                if line.rsplit("content")[1]:
                    target = line.split('"content":"')[1].split('","type')[0]    
                    temp.append(target)

        ### 데이터 저장 ###
        body = "".join(temp).replace('\\', "")        
        title = page.text.replace('\n', "")
        day = page.body.script.decode_contents().split('"display_date":"')[1].split(',"distributor"')[0].split('T')[0]
        company = '중앙일보'
        data_set[idx] = [day, title, body, company, urls[idx]]

    # 다른신문사는 제외 (exception 예방)
    else:
        continue
        
    
        
        
'''
데이터 내보내기 (뉴스본문)
'''
        
dataframe = pd.DataFrame(data_set).transpose()
dataframe.columns = ['date', 'title', 'body', 'company', 'url']
# 본문 내 특수문자 제거
dataframe['body'] = dataframe['body'].str.replace(pat=r'[^\w\s\.]', repl=r'', regex=True)
dataframe['body'] = dataframe['body'].str.replace(pat=r'["\xa0"]', repl = r'', regex=True)
# 한자 제거
dataframe['body'] = dataframe['body'].str.replace(pat=r'[\u3400-\u9FBF]', repl = r'', regex=True)
dataframe['body'] = dataframe['body'].str.replace(' b ', '')

dataframe = dataframe.reset_index()
dataframe.to_csv('중앙일보_{}to{}.csv'.format(first_date, end_date), encoding = "cp949")

20200201
20200202
20200203
20200204
20200205
